In [ ]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('..')
sys.setrecursionlimit(1000000)

from spectral import get_rgb, ndvi
import glob
import os

import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.metrics import fbeta_score
from sklearn.preprocessing import MinMaxScaler
from skimage.io import imread

from keras.models import Sequential
from keras.layers import *
from keras.callbacks import CSVLogger, ReduceLROnPlateau, ModelCheckpoint
from keras import backend as K
import paths

from rainforest.data import get_data, labels
from rainforest.preprocess import preprocess
from rainforest.models.resnet import ResNet50
from rainforest.models.densenet import DenseNet

In [ ]:
train_data = get_data(train=True)
val_data = get_data(train=False)

In [ ]:
batch_size=12
image_type = 'both' # 'jpg', 'tif' or 'both'
n_channels = 3 if image_type == 'jpg' else 5 if image_type == 'tif' else 8
input_shape = (n_channels, 64, 64)
input_shape = image_size if K.image_dim_ordering() == 'th' else tuple(reversed(input_shape))

In [ ]:
def preprocess_image1(img):
    img_rgb = get_rgb(img, [2, 1, 0])  # R-G-B
    rescaleimg = np.reshape(img_rgb, (-1, 1))
    scaler = MinMaxScaler(feature_range=(0, 255))
    rescaleimg = scaler.fit_transform(rescaleimg)  # .astype(np.float32)
    img_scaled = (np.reshape(rescaleimg, img_rgb.shape)) / 255.
    img_nir = get_rgb(img, [3, 2, 1])  # NIR-R-G
    img_nir_red = (img_nir[:, :, 0] - img_nir[:, :, 1]) / (img_nir[:, :, 0] + img_nir[:, :, 1] + np.finfo(float).eps)  # (NIR - RED) / (NIR + RED)
    img_nir_red = np.expand_dims(np.clip(img_nir_red, -1, 1), axis=2)
    img_nir_green = (img_nir[:, :, 2] - img_nir[:, :, 0]) / (img_nir[:, :, 2] + img_nir[:, :, 0] + np.finfo(float).eps)  # (GREEN - NIR) / (GREEN + NIR)
    img_nir_green = np.expand_dims(np.clip(img_nir_green, -1, 1), axis=2)

    return np.dstack((img_scaled, img_nir_red, img_nir_green))

In [ ]:
def data_generator(data_df, batch_size=32, target_size=(3, 256, 256), shuffle=True, augmentation=True,
                   mode='train', image_type='jpg'):
    n = len(data_df)
    use_tif = image_type in ['tif', 'both']
    use_jpg = image_type in ['jpg', 'both']
    cv2_image_shape = tuple(reversed(target_size[1:] if K.image_dim_ordering() == 'th' else target_size[:2]))
    while True:
        # Maybe shuffle
        data = data_df.sample(frac=1) if shuffle else data_df
        data = data.append(data, ignore_index=True)
        i = 0
        while i < n:
            X_batch = np.zeros((batch_size,)+target_size , dtype=np.float32)
            y_batch = np.zeros((batch_size, 17), dtype=np.uint8)
            
            for j in range(batch_size):
                img = data.iloc[i]
                images = []
                if use_tif:
                    tif_path = os.path.join(paths.DATA_FOLDER, mode+'-tif-v2', img.image_name+'.tif')
                    tif_image = imread(tif_path)
                    tif_image = preprocess_image1(tif_image)
                    images.append(tif_image)
                    
                if use_jpg:
                    jpg_path = os.path.join(paths.DATA_FOLDER, mode+'-jpg', img.image_name+'.jpg')
                    jpg_image = cv2.imread(jpg_path, cv2.IMREAD_COLOR)[:, :, :3]/255.
                    images.append(jpg_image)
                
                image = np.dstack(images)
                image = preprocess(image, target_size=cv2_image_shape, augmentation=augmentation,
                                   hflip=True, vflip=True, shift_x=3, shift_y=3, rot_range=5,
                                   dim_ordering=K.image_dim_ordering())
                X_batch[j] = image
                y_batch[j] = img[1:].values
                i += 1

            yield X_batch, y_batch

In [ ]:
def fb_score(beta=1, smooth=1e-6, threshold=0.2):
    axis = 1 if K.image_dim_ordering()  == 'th' else -1
    def fscore(y_true, y_pred):
        y_pred = y_pred > threshold
        recall = (K.sum(y_true * y_pred, axis=axis) + smooth) / (K.sum(y_true, axis=axis) + smooth)
        precision = (K.sum(y_true * y_pred, axis=axis) + smooth) / (K.sum(y_pred, axis=axis) + smooth)
        return K.mean( ((1+beta**2) * (precision*recall)+smooth) / (beta**2*precision+recall+smooth) )
    
    fscore.__name__ = 'F%d_score' % beta
    
    return fscore

In [ ]:
def resnet_like():
    model = ResNet50(input_shape=input_shape, classes=17, classification='sigmoid', layer1_filters=32)
    return model

In [ ]:
def vgg_like():
    model = Sequential([
        Conv2D(32, 3, activation='relu', kernel_initializer='he_normal', input_shape=input_shape
        BatchNormalization(axis=1),
        Conv2D(32, 3, activation='relu', kernel_initializer='he_normal'),
        BatchNormalization(axis=1),
        MaxPool2D(),

        Conv2D(64, 3, activation='relu', kernel_initializer='he_normal'),
        BatchNormalization(axis=1),
        Conv2D(64, 3, activation='relu', kernel_initializer='he_normal'),
        BatchNormalization(axis=1),
        MaxPool2D(),

        Conv2D(128, 3, activation='relu', kernel_initializer='he_normal'),
        BatchNormalization(axis=1),
        Conv2D(128, 3, activation='relu', kernel_initializer='he_normal'),
        BatchNormalization(axis=1),
        MaxPool2D(),

        Flatten(),
        Dense(1024, activation='relu', kernel_initializer='he_normal'),
        BatchNormalization(),
        Dense(17, activation='sigmoid')
    ])
    
    return model


In [ ]:
model = DenseNet(input_shape=input_shape, depth=121, bottleneck=True, reduction=0.5, weight_decay=0, weights=None, classes=17, activation='sigmoid')

model.summary()
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', fb_score(beta=2)])

In [ ]:
train_gen = data_generator(train_data, batch_size=batch_size, target_size=input_size, shuffle=True, augmentation=True,
                           mode='train', image_type='both')
val_gen = data_generator(val_data, batch_size=batch_size, target_size=input_size, shuffle=False, augmentation=False,
                        mode='train', image_type='both')

In [ ]:
csv_logger = CSVLogger('log.csv')
lr_plateau = ReduceLROnPlateau(monitor='val_loss', patience=3, verbose=1, factor=0.25)
checkpoint = ModelCheckpoint(filepath='E:/Models/brainforest/densenet_64x64_tifjpg.hdf5', verbose=1, save_best_only=True)

In [ ]:
train_steps = len(train_data) // batch_size
val_steps = len(val_data) // batch_size
model.fit_generator(train_gen, train_steps, epochs=50, callbacks=[csv_logger, lr_plateau, checkpoint],
                    validation_data=val_gen, validation_steps=val_steps)

1189/2361 [==============>...............] - ETA: 4078s - loss: 0.1801 - acc: 0.9271 - F2_score: 0.8282

In [ ]:
from keras.models import load_model
model = load_model('E:/Models/brainforest/densenet_64x64_tifjpg.hdf5', custom_objects={'F2_score': fb_score(beta=2)})

def strip_labels(gen):
    while True:
        imgs, _ = next(gen)
        yield imgs

val_steps = int(np.ceil(len(val_data) // batch_size)) + 1
val_gen = strip_labels(data_generator(val_data, batch_size=batch_size, target_size=input_size, shuffle=False, mode='train', image_type='both'))
preds = model.predict_generator(val_gen, val_steps, verbose=1)
preds = preds[:len(val_data)]

In [ ]:
for threshold in np.arange(0.1, 0.3, 0.02):
    y_true = val_data.iloc[:, 1:].values
    y_pred =  preds > threshold
    print threshold, 'f2 score:', fbeta_score(y_true, y_pred, 2, average='samples')

In [ ]:
test_files = glob.glob(os.path.join(paths.DATA_FOLDER, 'test-jpg', '*.jpg'))
test_files = [os.path.basename(os.path.splitext(f)[0]) for f in test_files]
test_data = pd.DataFrame(test_files, columns=['image_name'])
test_data['bogus_label'] = np.zeros(len(test_files))

In [ ]:
test_data.head()

In [ ]:
test_steps = int(np.ceil(len(test_data) // batch_size)) + 1
test_gen = strip_labels(data_generator(test_data, batch_size=batch_size, target_size=input_size, shuffle=False, subfolder='test-jpg'))
preds = model.predict_generator(test_gen, test_steps)
preds = preds[:len(test_data)]

In [ ]:
tpreds = preds > 0.16

In [ ]:
with open('submission2.csv', 'w') as file:
    file.write('image_name,tags\n')
    for img, pred in zip(test_files, tpreds):
        indices = np.flatnonzero(pred)
        tags = ' '.join([labels[i] for i in indices])
        file.write('%s,%s\n' % (img, tags))

In [ ]:
from sklearn.externals import joblib
joblib.dump(preds, 'multilabel2405_128x128.pkl')

In [ ]:
preds = (joblib.load('multilabel2405_128x128.pkl') + joblib.load('multilabel2405.pkl')) / 2

In [ ]:
tpreds = preds > 0.16

In [ ]:
with open('submission3.csv', 'w') as file:
    file.write('image_name,tags\n')
    for img, pred in zip(test_files, tpreds):
        indices = np.flatnonzero(pred)
        tags = ' '.join([labels[i] for i in indices])
        file.write('%s,%s\n' % (img, tags))